<a href="https://colab.research.google.com/github/sSteam-cube/P_345-P_343-Laboratory-Resources/blob/main/Milikans_experiment/Milikans_oil_drop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dynamic Method


In [ ]:
import numpy as np
import pandas as pd
import statistics as stats

# Constants
C = 190.13
D = 9.04e-9
zeta = 4.06e-8
L = 1.0e-3  # distance between lines in m


dynamic_data =  [
    ([15.8, 16, 16.5, 19.5, 18.4], [8.1, 7.5, 8.1, 7.7, 7.6], 294),  # Droplet 1
    ([16.2, 17.0, 16.3, 16.8, 17.4], [4.3, 4.2, 4.2, 4.2, 4.1], 140),  # Droplet 2
    ([17.4, 15.6, 15.8, 15.7, 15.5], [2.1, 2.1, 2.2, 2.1, 2.4], 141),  # Droplet 3
    ([21.9, 20.2, 20.6, 19.6, 20.8], [7.0, 7.1, 7.1, 6.4, 7.1], 274),  # Droplet 4
    ([17.6, 18.0, 18.0, 16.2], [4.8, 4.8, 4.7, 5.2], 140),             # Droplet 5
    ([5.4, 5.4, 5.4, 5.7, 5.7], [4.4, 4.7, 4.3, 4.0, 4.5], 300)        # Droplet 6
]

# Compute means
data = [
    [stats.mean(pos), stats.mean(vel), val]
    for pos, vel, val in dynamic_data
]

results = []

for tf, tr, V in data:
    vf = L / tf                      # free-fall velocity
    xi = D * vf                      # ξ
    r = -zeta + np.sqrt(zeta**2 + xi)  # radius
    r3 = r**3
    T = 1 + (tf / tr)
    ne = (C * T * r3) / V            # charge
    results.append([xi, r, r3, T, ne])

# Make a table
df = pd.DataFrame(results, columns=["ξ (m²)", "r (m)", "r³ (m³)", "T", "ne (C)"])
print(df)

# --- Continue from your previous code ---

# Extract ne column
ne_values = df["ne (C)"].to_numpy()

# Step 1: minimum value of ne
ne_min = ne_values.min()

# Step 2–4: build the table
table = []
for ne in ne_values:
    ratio = ne / ne_min
    n_eff = int(round(ratio))  # nearest integer
    e_val = ne / n_eff
    table.append([ne, ratio, n_eff, e_val])

# Step 5: average e
avg_e = np.mean([row[3] for row in table])

# Create DataFrame for clarity
df_dynamic = pd.DataFrame(
    table,
    columns=["ne (C)", "ne / lowest", "nearest integer (n_eff)", "ne / n_eff"]
)

print("Dynamic Method Data:\n", df_dynamic)
print("\nAverage elementary charge estimate =", avg_e)

         ξ (m²)         r (m)       r³ (m³)         T        ne (C)
0  5.243619e-13  6.846657e-07  3.209487e-19  3.210256  6.663137e-19
1  5.400239e-13  6.953839e-07  3.362589e-19  4.985714  2.276795e-18
2  5.650000e-13  7.121605e-07  3.611883e-19  8.339450  4.061651e-18
3  4.384093e-13  6.227684e-07  2.415348e-19  3.971182  6.655790e-19
4  5.180516e-13  6.803022e-07  3.148514e-19  4.579487  1.958146e-18
5  1.637681e-12  1.239763e-06  1.905531e-18  2.260274  2.729647e-18
Dynamic Method Data:
          ne (C)  ne / lowest  nearest integer (n_eff)    ne / n_eff
0  6.663137e-19     1.001104                        1  6.663137e-19
1  2.276795e-18     3.420773                        3  7.589315e-19
2  4.061651e-18     6.102432                        6  6.769418e-19
3  6.655790e-19     1.000000                        1  6.655790e-19
4  1.958146e-18     2.942019                        3  6.527153e-19
5  2.729647e-18     4.101162                        4  6.824118e-19

Average elementary charge

# Balancing Method

In [ ]:
import numpy as np

# This script combines the full calculation and analysis for the
# Millikan oil drop experiment's Balancing Method.

# --- Constants from the Millikan Oil Drop Experiment Manual ---
L = 1.0e-3      # Distance the droplet falls in meters
C = 190.13      # Pre-calculated constant
D = 9.04e-9     # Pre-calculated constant
zeta = 4.06e-8  # Pre-calculated constant

# --- Experimental Data from Handwritten Notes ---
free_fall_times = [
    [15.6, 15.0, 17.1, 16.1, 16.0],
    [9.0, 8.5, 8.5, 9.5, 9.1],
    [18.0, 20.1, 19.1, 20.6, 19.5],
    [13.5, 12.5, 13.7, 12.7, 12.1],
    [12.5, 13.3, 12.6, 12.8, 12.9]
]

balancing_voltages = [
    [348, 332, 315, 333, 325],
    [394, 387, 404, 392, 399],
    [220, 212, 200, 207, 211],
    [470, 474, 477, 475, 477],
    [453, 449, 452, 455, 447]
]

# --- Part 1: Calculation of Total Charge (ne) on Each Droplet ---

print("## Part 1: Balancing Method Calculation")
print("-" * 95)
print(f"{'Droplet No.':<15}{'ξ (=Dv_f)':<20}{'r (=-ζ+sqrt(ζ²+ξ))':<25}{'r³':<20}{'ne (=Cr³/Vb)':<20}")
print("-" * 95)

ne_values = []
table_data_for_user = [] # List to store data for the user's table

for i in range(len(free_fall_times)):
    droplet_num = i + 1

    t_f = np.mean(free_fall_times[i])
    V_b = np.mean(balancing_voltages[i])
    v_f = L / t_f
    xi = D * v_f
    r = -zeta + np.sqrt(zeta**2 + xi)
    r_cubed = r**3
    ne = (C * r_cubed) / V_b

    ne_values.append(ne)
    # Store the intermediate results needed for the user's table
    table_data_for_user.append({'droplet': droplet_num, 't_f': t_f, 'v_f': v_f, 'V_b': V_b})

    print(f"{droplet_num:<15}"
          f"{xi:<20.3e}"
          f"{r:<25.3e}"
          f"{r_cubed:<20.3e}"
          f"{ne:<20.3e}")
print("-" * 95)

# --- Part 2: Analysis of Quantized Charge to Find 'e' ---

print("\n## Part 2: Analysis of Charge to Determine 'e'")

lowest_ne = np.min(ne_values)
e_values = []

print("-" * 95)
print(f"{'Droplet No.':<15}{'ne':<20}{'ne / lowest_ne':<25}{'n_eff (integer)':<20}{'e = ne/n_eff':<20}")
print("-" * 95)

for i, ne in enumerate(ne_values):
    ratio = ne / lowest_ne
    n_eff = int(np.round(ratio))
    e = ne / n_eff
    e_values.append(e)

    print(f"{i+1:<15}"
          f"{ne:<20.3e}"
          f"{ratio:<25.3f}"
          f"{n_eff:<20}"
          f"{e:<20.3e}")
print("-" * 95)

average_e = np.mean(e_values)
print(f"\n🧪 **Final Result**: The average experimental value for 'e' is: **{average_e:.4e} C**")




## Part 1: Balancing Method Calculation
-----------------------------------------------------------------------------------------------
Droplet No.    ξ (=Dv_f)           r (=-ζ+sqrt(ζ²+ξ))       r³                  ne (=Cr³/Vb)        
-----------------------------------------------------------------------------------------------
1              5.664e-13           7.131e-07                3.626e-19           2.085e-19           
2              1.013e-12           9.669e-07                9.040e-19           4.349e-19           
3              4.645e-13           6.422e-07                2.648e-19           2.398e-19           
4              7.008e-13           7.975e-07                5.072e-19           2.032e-19           
5              7.051e-13           8.001e-07                5.122e-19           2.158e-19           
-----------------------------------------------------------------------------------------------

## Part 2: Analysis of Charge to Determine 'e'
------------------

In [ ]:
# --- Part 3: Values for Your Observation Table ---

print("\n\n## Part 3: Data for Your Observation Table 📝")
print("Here are the specific values you requested to fill in your table:")

for data in table_data_for_user:
    print("-" * 55)
    print(f"**Droplet {data['droplet']}**")
    print(f"  - Mean Free-fall Time (t_f):      {data['t_f']:.2f} s")
    print(f"  - Mean Free-fall Velocity (v_f):  {data['v_f']:.4e} m/s")
    print(f"  - Final Balancing Voltage (V_b):  {data['V_b']:.2f} V")
print("-" * 55)



## Part 3: Data for Your Observation Table 📝
Here are the specific values you requested to fill in your table:
-------------------------------------------------------
**Droplet 1**
  - Mean Free-fall Time (t_f):      15.96 s
  - Mean Free-fall Velocity (v_f):  6.2657e-05 m/s
  - Final Balancing Voltage (V_b):  330.60 V
-------------------------------------------------------
**Droplet 2**
  - Mean Free-fall Time (t_f):      8.92 s
  - Mean Free-fall Velocity (v_f):  1.1211e-04 m/s
  - Final Balancing Voltage (V_b):  395.20 V
-------------------------------------------------------
**Droplet 3**
  - Mean Free-fall Time (t_f):      19.46 s
  - Mean Free-fall Velocity (v_f):  5.1387e-05 m/s
  - Final Balancing Voltage (V_b):  210.00 V
-------------------------------------------------------
**Droplet 4**
  - Mean Free-fall Time (t_f):      12.90 s
  - Mean Free-fall Velocity (v_f):  7.7519e-05 m/s
  - Final Balancing Voltage (V_b):  474.60 V
------------------------------------------------

In [ ]:
np.array(e_values)/1.602e-19

array([1.30177959, 1.35742547, 1.49672791, 1.26842303, 1.34732168])